## Configuration

**Note**: This notebook requires Python 3.11+ for the full krl-model-zoo package. 

If you're using Python 3.9, set `USE_MOCK_DATA = True` below to test with synthetic data.

In [2]:
USE_MOCK_DATA = False  # Using available enterprise/professional tier connectors

In [2]:
# Install KRL packages from local workspace (run once, then restart kernel)
import sys
import subprocess
import IPython

krl_packages = [
    "/Users/bcdelo/Documents/GitHub/KRL/Private IP/krl-data-connectors",
    "/Users/bcdelo/Documents/GitHub/KRL/Private IP/krl-model-zoo"
]

for pkg_path in krl_packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-e", pkg_path, "-q"])
    
print("✅ KRL packages installed successfully")
print("🔄 Restarting kernel to load packages...")

# Restart kernel
IPython.Application.instance().kernel.do_shutdown(True)

✅ KRL packages installed successfully
🔄 Restarting kernel to load packages...


{'status': 'ok', 'restart': True}

# Healthcare Causal Analysis with GRU Causal Gates

**Domain**: Healthcare Policy Analysis  
**Model**: GRU with Causal Recurrence Gates (Sprint 7 Enhancement)  
**Data Sources**: 
- County Health Rankings (Professional Tier) - Health outcomes
- Census ACS Detailed (Professional Tier) - Demographics & socioeconomic data
- BRFSS (Professional Tier) - Behavioral health data

**Focus**: Causal relationships in health outcomes across demographics

## Overview

This notebook demonstrates the **GRU Causal Recurrence Gates** enhancement from Sprint 7, which enables multi-domain causal analysis by constraining gate activations according to a directed acyclic graph (DAG).

**Causal DAG**:
```
Healthcare Outcomes ← Social Determinants ← Economic Factors
        ↑                      ↑                    ↑
    (mortality,           (education,         (income,
     morbidity,           housing,            employment,
     life expectancy)     food security)       insurance)
```

**Key Features**:
- DAG-constrained gate activations prevent spurious correlations
- Causal masking on feature dimensions (not time)
- Health disparity metrics across race/ethnicity, income, geography
- Professional-tier obfuscated model components

**Data Coverage**: County-level health outcomes (2015-2023) merged with demographic/economic indicators

## Setup and Imports

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Conditional imports based on USE_MOCK_DATA flag
if not USE_MOCK_DATA:
    import torch
    import torch.nn as nn
    # Professional tier connectors (imported from package)
    from krl_data_connectors.professional import (
        CountyHealthRankingsConnector,
        CensusACSDetailedConnector,
        BRFSSConnector
    )
    # Model zoo
    from krl_model_zoo.time_series.gru import GRUModel
    print("✓ Using real connectors and models")
    print("  - CountyHealthRankingsConnector (health outcomes)")
    print("  - CensusACSDetailedConnector (demographics/socioeconomic)")
    print("  - BRFSSConnector (behavioral health)")
else:
    print("✓ Using mock data mode")
    print("  For full functionality, set USE_MOCK_DATA = False and install krl packages")

# Set random seeds for reproducibility
np.random.seed(42)
if not USE_MOCK_DATA:
    torch.manual_seed(42)

# Configure plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)


ImportError: cannot import name 'CensusACSDetailedConnector' from 'krl_data_connectors.professional.demographic.census_acs_detailed' (/Users/bcdelo/Documents/GitHub/KRL/Private IP/krl-data-connectors/src/krl_data_connectors/professional/demographic/census_acs_detailed.py)

## 1. Data Collection

Fetch healthcare and demographic data from CDC and Census APIs.

In [ ]:
# Initialize connectors
chr_connector = CountyHealthRankingsConnector()
census = CensusACSDetailedConnector()

# Fetch County Health Rankings data (2015-2023)
# Provides: mortality rates, life expectancy, chronic disease prevalence, 
#           healthcare access, behavioral risk factors by county
print("Fetching County Health Rankings data...")
health_data = chr_connector.get_health_outcomes(
    years=range(2015, 2024),
    states='all',  # All US counties
    measures=[
        'premature_death',        # Years of potential life lost
        'poor_or_fair_health',    # % reporting poor/fair health
        'poor_physical_health',   # Average unhealthy physical days
        'poor_mental_health',     # Average unhealthy mental days
        'adult_obesity',          # % adults with BMI >= 30
        'diabetes_prevalence',    # % with diagnosed diabetes
        'excessive_drinking',     # % binge or heavy drinking
        'uninsured'              # % without health insurance
    ]
)

print(f"✓ County Health Rankings data shape: {health_data.shape}")
print(f"  Columns: {health_data.columns.tolist()[:8]}")

In [ ]:
# Fetch Census ACS demographic and socioeconomic data (2015-2023)
# Variables: income, education, housing, employment, food security, race/ethnicity
print("\nFetching Census ACS detailed data...")
demographic_data = census.get_detailed_acs_data(
    geography='county',
    years=range(2015, 2024),
    variables=[
        'B19013_001E',  # Median household income
        'B17001_002E',  # Poverty: Population below poverty level
        'B15003_022E',  # Education: Bachelor's degree
        'B15003_001E',  # Education: Total population 25+
        'B25014_005E',  # Housing: Overcrowded (>1.5 persons/room)
        'B23025_005E',  # Employment: Unemployed
        'B22003_002E',  # Food stamps/SNAP recipients
        'B27001_004E',  # Health insurance: With coverage
        'B27001_001E',  # Health insurance: Total population
        'B02001_002E',  # Race: White alone
        'B02001_003E',  # Race: Black/African American alone
        'B03003_003E',  # Hispanic or Latino
    ]
)

# Calculate derived percentages
demographic_data['poverty_rate'] = (demographic_data['B17001_002E'] / 
                                     demographic_data['B17001_002E'].sum()) * 100
demographic_data['bachelors_pct'] = (demographic_data['B15003_022E'] / 
                                      demographic_data['B15003_001E']) * 100
demographic_data['unemployment_rate'] = (demographic_data['B23025_005E'] / 
                                          demographic_data['B23025_005E'].sum()) * 100
demographic_data['snap_pct'] = (demographic_data['B22003_002E'] / 
                                 demographic_data['B22003_002E'].sum()) * 100
demographic_data['uninsured_pct'] = ((demographic_data['B27001_001E'] - 
                                       demographic_data['B27001_004E']) / 
                                      demographic_data['B27001_001E']) * 100

print(f"✓ Census ACS data shape: {demographic_data.shape}")
print(f"  Derived variables: poverty_rate, bachelors_pct, unemployment_rate, snap_pct, uninsured_pct")

## 2. Data Preprocessing and Integration

In [ ]:
# Merge health and demographic data on county and year
merged_data = pd.merge(
    health_data,
    demographic_data,
    on=['county_fips', 'year'],
    how='inner'
)

print(f"Merged data shape: {merged_data.shape}")
print(f"\nSample data:")
print(merged_data.head())

In [ ]:
# Define causal variable groupings based on domain knowledge
# These correspond to the causal DAG structure

# Economic factors (root causes in DAG)
economic_vars = ['MEDIAN_INCOME', 'POVERTY_RATE', 'UNEMPLOYMENT_RATE', 'INSURANCE_COVERAGE']

# Social determinants (intermediate causes)
social_vars = ['EDUCATION_BACHELORS_PCT', 'HOUSING_CROWDING', 'FOOD_STAMP_PCT']

# Health outcomes (effects in DAG)
health_outcome_vars = ['MORT_ALL_CAUSE', 'MORT_HEART_DISEASE', 'MORT_CANCER', 
                       'PREV_DIABETES', 'PREV_HYPERTENSION', 'HEALTHCARE_ACCESS']

# Combine in causal order: economic → social → health
feature_columns = economic_vars + social_vars + health_outcome_vars
n_variables = len(feature_columns)

print(f"Total features: {n_variables}")
print(f"  Economic: {len(economic_vars)}")
print(f"  Social: {len(social_vars)}")
print(f"  Health outcomes: {len(health_outcome_vars)}")

In [ ]:
# Create sequences for time series modeling
# Each sequence: 5 years of history → predict next year

def create_sequences(data, seq_length=5):
    """Create sliding window sequences for time series prediction."""
    X, y = [], []
    
    # Group by county to maintain temporal coherence
    for county in data['county_fips'].unique():
        county_data = data[data['county_fips'] == county].sort_values('year')
        county_features = county_data[feature_columns].values
        
        # Create sequences
        for i in range(len(county_features) - seq_length):
            X.append(county_features[i:i+seq_length])
            # Predict mortality rate for next year
            y.append(county_features[i+seq_length, 0])  # MORT_ALL_CAUSE
    
    return np.array(X), np.array(y)

X_sequences, y_targets = create_sequences(merged_data, seq_length=5)

print(f"Sequence shape: {X_sequences.shape}")  # (n_samples, seq_length, n_features)
print(f"Target shape: {y_targets.shape}")      # (n_samples,)

In [ ]:
# Normalize features
scaler = StandardScaler()

# Reshape for scaling
n_samples, seq_length, n_features = X_sequences.shape
X_flat = X_sequences.reshape(-1, n_features)

# Fit and transform
X_scaled = scaler.fit_transform(X_flat)
X_scaled = X_scaled.reshape(n_samples, seq_length, n_features)

# Split into train/val/test
X_train, X_temp, y_train, y_temp = train_test_split(
    X_scaled, y_targets, test_size=0.3, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

print(f"Train: {X_train.shape[0]} samples")
print(f"Val: {X_val.shape[0]} samples")
print(f"Test: {X_test.shape[0]} samples")

## 3. Define Causal DAG

The causal DAG encodes domain knowledge about healthcare determinants:
- Economic factors influence social determinants
- Social determinants influence health outcomes
- No reverse causation (enforced by DAG structure)

In [ ]:
# Define causal DAG as adjacency matrix
# Entry [i,j] = 1 means variable i causes variable j

causal_dag = np.zeros((n_variables, n_variables))

# Economic factors → Social determinants
for i in range(len(economic_vars)):
    for j in range(len(economic_vars), len(economic_vars) + len(social_vars)):
        causal_dag[i, j] = 1

# Social determinants → Health outcomes
for i in range(len(economic_vars), len(economic_vars) + len(social_vars)):
    for j in range(len(economic_vars) + len(social_vars), n_variables):
        causal_dag[i, j] = 1

# Economic factors → Health outcomes (direct effects)
for i in range(len(economic_vars)):
    for j in range(len(economic_vars) + len(social_vars), n_variables):
        causal_dag[i, j] = 1

# Visualize DAG
plt.figure(figsize=(10, 8))
sns.heatmap(causal_dag, annot=False, cmap='Blues', cbar_kws={'label': 'Causal Edge'})
plt.xlabel('Effect Variable')
plt.ylabel('Cause Variable')
plt.title('Healthcare Causal DAG\nEconomic → Social → Health')
plt.tight_layout()
plt.show()

print(f"DAG shape: {causal_dag.shape}")
print(f"Total causal edges: {causal_dag.sum():.0f}")

## 4. Initialize GRU Model with Causal Gates

**Sprint 7 Enhancement**: `use_causal_gates=True` activates DAG-constrained recurrence.

In [ ]:
# Model hyperparameters
input_size = n_variables
hidden_size = 128
num_layers = 2
output_size = 1  # Predicting mortality rate

# Initialize model with causal gates (Professional tier enhancement)
model = GRUModel(
    input_size=input_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    output_size=output_size,
    use_causal_gates=True,       # Sprint 7 enhancement
    n_variables=n_variables,      # Required for causal masking
    causal_dag=causal_dag         # DAG structure
)

print(f"Model initialized with causal gates")
print(f"Input size: {input_size}")
print(f"Hidden size: {hidden_size}")
print(f"Causal edges: {causal_dag.sum():.0f}")
print(f"\nModel architecture:")
print(model)

## 5. Training

In [ ]:
# Training configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

# Convert to tensors
X_train_t = torch.FloatTensor(X_train).to(device)
y_train_t = torch.FloatTensor(y_train).unsqueeze(1).to(device)
X_val_t = torch.FloatTensor(X_val).to(device)
y_val_t = torch.FloatTensor(y_val).unsqueeze(1).to(device)

print(f"Training on device: {device}")

In [ ]:
# Training loop
num_epochs = 50
batch_size = 32
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    
    # Mini-batch training
    for i in range(0, len(X_train_t), batch_size):
        batch_X = X_train_t[i:i+batch_size]
        batch_y = y_train_t[i:i+batch_size]
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        epoch_loss += loss.item()
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_t)
        val_loss = criterion(val_outputs, y_val_t).item()
    
    train_losses.append(epoch_loss / (len(X_train_t) / batch_size))
    val_losses.append(val_loss)
    
    scheduler.step(val_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}] "
              f"Train Loss: {train_losses[-1]:.4f} "
              f"Val Loss: {val_losses[-1]:.4f}")

In [ ]:
# Plot training history
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss', alpha=0.8)
plt.plot(val_losses, label='Validation Loss', alpha=0.8)
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('GRU with Causal Gates - Training History')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Evaluation and Health Disparity Analysis

In [ ]:
# Test set predictions
model.eval()
X_test_t = torch.FloatTensor(X_test).to(device)
y_test_t = torch.FloatTensor(y_test).unsqueeze(1).to(device)

with torch.no_grad():
    y_pred = model(X_test_t).cpu().numpy()

y_test_np = y_test_t.cpu().numpy()

# Calculate metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test_np, y_pred)
rmse = np.sqrt(mean_squared_error(y_test_np, y_pred))
r2 = r2_score(y_test_np, y_pred)

print(f"Test Set Performance:")
print(f"  MAE: {mae:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  R²: {r2:.4f}")

In [ ]:
# Prediction vs actual scatter plot
plt.figure(figsize=(8, 8))
plt.scatter(y_test_np, y_pred, alpha=0.5, s=20)
plt.plot([y_test_np.min(), y_test_np.max()], 
         [y_test_np.min(), y_test_np.max()], 
         'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual Mortality Rate')
plt.ylabel('Predicted Mortality Rate')
plt.title(f'GRU Causal Gates - Predictions\nR² = {r2:.3f}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Causal Interpretation

Analyze how the causal DAG structure influenced predictions.

In [ ]:
# Extract causal gate weights (if available in model)
# This demonstrates the Professional-tier enhancement

try:
    # Access causal gate parameters
    causal_weights = model.gru.causal_gates.causal_mask.detach().cpu().numpy()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(causal_weights, annot=False, cmap='RdYlGn', center=0,
                cbar_kws={'label': 'Gate Activation Strength'})
    plt.xlabel('Target Variable')
    plt.ylabel('Source Variable')
    plt.title('Learned Causal Gate Weights\n(Constrained by DAG Structure)')
    plt.tight_layout()
    plt.show()
    
    print("Causal gate analysis:")
    print(f"  Strongest economic → health path: {causal_weights[:4, -6:].mean():.3f}")
    print(f"  Strongest social → health path: {causal_weights[4:7, -6:].mean():.3f}")
    print(f"  Economic → social influence: {causal_weights[:4, 4:7].mean():.3f}")
    
except AttributeError:
    print("Causal gate weights not directly accessible (obfuscated Professional tier code)")

## 8. Summary

This notebook demonstrated:

1. **Multi-domain data integration**: CDC health outcomes + Census demographics
2. **Causal DAG structure**: Economic → Social → Health relationships
3. **GRU Causal Gates**: Sprint 7 enhancement for DAG-constrained recurrence
4. **Health disparity analysis**: Predictions informed by causal structure

**Professional Tier Value**:
- Access to CDC_Full and Census_ACS_Detailed connectors
- GRU Causal Gates enhancement (obfuscated proprietary code)
- Prevents spurious correlations through causal constraints

**Next Steps**:
- Extend to multi-outcome prediction (mortality, morbidity, access)
- Incorporate additional social determinants (education quality, environmental factors)
- Stratified analysis by race/ethnicity and income quintiles
- Counterfactual analysis for policy interventions